In [116]:
from math import pi,cos,sin
import os
import pandas as pd
import time
import pickle
from multiprocessing import Pool
import multiprocessing as mp
from joblib import Parallel, delayed
from queue import PriorityQueue
import numpy as np
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()
inputs = myList



In [43]:
os.chdir('/home/lucasperea/Documents/Diffusion_MOF/')

In [161]:
# %store -r path_dic
%store path_dic
len(path_dic)

Stored 'path_dic' (dict)


1249

In [156]:
def funclon(x):
    y = x+2+5
    return y

if __name__ == "__main__":
    

    pool = mp.Pool(mp.cpu_count())

    result = pool.map_async(func=funclon, iterable=list(range(100)), callback=get_result)
    pool.close()
    pool.join()

In [157]:
def VolDist(j):
    temporal_dic = {}
    nt2_name = os.listdir('./Nt2Files_MOF/')[876:][j]
    print(nt2_name)

    vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt = CellLocation(nt2_name)
    G, connection_cell, larger_node_dic = GraphCell(nt2_name, pos_order)

    final, key_plus = GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh)
    Y, Y1 = TransfMatrix(vector_values_latt)
    final2 = distance(final,3, Y, Y1)



    neigh_dic = {}
    for n, nbrs in final2.adj.items():
        neigh = []
        for nbr, eattr in nbrs.items():
            neigh.append(nbr)


        neigh_dic[str(n)] = neigh

    all_path = []
    total_path = []
    special_list = list(range(13))

    larger_node = []
    name_larger_node = []
    for node in final.nodes:
        larger_node.append(final.nodes[node]['rad_max_sph'])
        name_larger_node.append(node)

    node_largest = name_larger_node[np.argmax(larger_node)]
    node_smallest = name_larger_node[np.argmin(larger_node)]
    node_path = node_largest #mat_pr_dic[nt2_name.split('.')[0]]

    for location in special_list:
        parents = {}

        start = node_path + key_plus*location

        target = key_plus*pos_order['000'] + node_path

        D, P = dijkstra(start, target, parents, final2, 0)
        try:
            path = backpedal(start, target, P)
        except:
            path = [node_smallest, neigh_dic[str(node_smallest)][0]]

        distance_path = []
        for i in range(len(path)-1):

            distance_path.append(final2[path[i]][path[i+1]]['distance'])


        all_path.append(path)
        total_path.append(sum(distance_path))  


    total_vol = []
    for ind_path in all_path:
        volume = []
        for i in range(len(ind_path) - 1):
            volume.append((final2[ind_path[i]][ind_path[i+1]]['distance'] * (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi))
    #                 ((final2[ind_path[i]][ind_path[i+1]]['distance'] - final2.nodes[ind_path[i]]['rad_max_sph'] - 
    #                   final2.nodes[ind_path[i+1]]['rad_max_sph'])*
    #                  (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi) + 
    #                 (final2.nodes[ind_path[i]]['rad_max_sph']**3)*(4/3)*pi +
    #                 (final2.nodes[ind_path[i+1]]['rad_max_sph']**3)*(4/3)*pi))
        total_vol.append(sum(volume))

    path_dic_mat = {}

    for j in range(len(list(pos_order.keys())[0:13])):
        path_dic_mat[str(list(pos_order.keys())[j])] = round(total_vol[j]/total_path[j], 2)


#     path_dic[nt2_name.split('.')[0]] = path_dic_mat
    
    temporal_dic[nt2_name.split('.')[0]] = path_dic_mat
    return temporal_dic

In [120]:
def get_result(result):
    global results
    results.append(result)

In [ ]:
path_dic = {}
for file in os.listdir('./Nt2Files_MOF/'):

    nt2_name = file


    vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt = CellLocation(nt2_name)
    G, connection_cell, larger_node_dic = GraphCell(nt2_name, pos_order)

    final, key_plus = GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh)
    Y, Y1 = TransfMatrix(vector_values_latt)
    final2 = distance(final,3, Y, Y1)



    neigh_dic = {}
    for n, nbrs in final2.adj.items():
        neigh = []
        for nbr, eattr in nbrs.items():
            neigh.append(nbr)


        neigh_dic[str(n)] = neigh

    all_path = []
    total_path = []
    special_list = list(range(13))

    larger_node = []
    name_larger_node = []
    for node in final.nodes:
        larger_node.append(final.nodes[node]['rad_max_sph'])
        name_larger_node.append(node)

    node_largest = name_larger_node[np.argmax(larger_node)]
    node_smallest = name_larger_node[np.argmin(larger_node)]
    node_path = node_largest #mat_pr_dic[nt2_name.split('.')[0]]

    for location in special_list:
        parents = {}

        start = node_path + key_plus*location

        target = key_plus*pos_order['000'] + node_path

        D, P = dijkstra(start, target, parents, final2, 0)
        try:
            path = backpedal(start, target, P)
        except:
            path = [node_smallest, neigh_dic[str(node_smallest)][0]]

        distance_path = []
        for i in range(len(path)-1):

            distance_path.append(final2[path[i]][path[i+1]]['distance'])


        all_path.append(path)
        total_path.append(sum(distance_path))  


    total_vol = []
    for ind_path in all_path:
        volume = []
        for i in range(len(ind_path) - 1):
        
                    volume.append(final2[ind_path[i]][ind_path[i+1]]['distance']*
                     (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi)
                                     
#                      ((final2[ind_path[i]][ind_path[i+1]]['distance']- final2.nodes[ind_path[i]]['rad_max_sph'] - 
#                       final2.nodes[ind_path[i+1]]['rad_max_sph'])*
#                      (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi) + 
#                     (final2.nodes[ind_path[i]]['rad_max_sph']**3)*(4/3)*pi +
#                     (final2.nodes[ind_path[i+1]]['rad_max_sph']**3)*(4/3)*pi))
#              volume.append((final2[ind_path[i]][ind_path[i+1]]['distance'] * (final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi))
        total_vol.append(sum(volume))

    path_dic_mat = {}

    for j in range(len(list(pos_order.keys())[0:13])):
        path_dic_mat[str(list(pos_order.keys())[j])] = round(total_vol[j]/total_path[j], 2)


    path_dic[nt2_name.split('.')[0]] = path_dic_mat

Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 4737 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8629 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 reached
Target vertex: 8794 

In [ ]:
%store path_dic

In [164]:
len(path_dic)

1249

In [110]:
for i in range(len(all_path[4])-1):
    print(final2.nodes[all_path[4][i]]['rad_max_sph'],final2[all_path[4][i]][all_path[4][i+1]]['rad_max_sph'], final2.nodes[all_path[4][i]]['rad_max_sph'])

7.70855 7.52908 7.70855
7.52908 5.7993 7.52908
5.7993 3.00974 5.7993
5.7993 5.7993 5.7993
7.52908 7.52908 7.52908


In [105]:
path_dic

{'VABJIS_clean': {'-1-1-1': 0.5,
  '-1-10': 0.5,
  '-1-11': 0.5,
  '-10-1': 0.5,
  '-100': 0.5,
  '-101': 0.5,
  '-11-1': 0.5,
  '-110': 0.5,
  '-111': 0.5,
  '0-1-1': 0.5,
  '0-10': 0.5,
  '0-11': 0.5,
  '00-1': 29.19},
 'VEXYON_charged': {'-1-1-1': 26.23,
  '-1-10': 24.22,
  '-1-11': 26.23,
  '-10-1': 24.22,
  '-100': 58.68,
  '-101': 24.22,
  '-11-1': 26.23,
  '-110': 24.22,
  '-111': 26.23,
  '0-1-1': 24.22,
  '0-10': 58.68,
  '0-11': 24.22,
  '00-1': 58.68}}

In [101]:
path_dic

{'VABJIS_clean': {'-1-1-1': 0.5,
  '-1-10': 0.5,
  '-1-11': 0.5,
  '-10-1': 0.5,
  '-100': 0.5,
  '-101': 0.5,
  '-11-1': 0.5,
  '-110': 0.5,
  '-111': 0.5,
  '0-1-1': 0.5,
  '0-10': 0.5,
  '0-11': 0.5,
  '00-1': 29.19},
 'VEXYON_charged': {'-1-1-1': 553.92,
  '-1-10': 470.97,
  '-1-11': 553.92,
  '-10-1': 470.97,
  '-100': 1583.26,
  '-101': 470.97,
  '-11-1': 553.92,
  '-110': 470.96,
  '-111': 553.92,
  '0-1-1': 471.3,
  '0-10': 1583.26,
  '0-11': 470.97,
  '00-1': 1583.26}}

In [92]:
path_dic

{'VABJIS_clean': {'-1-1-1': 0.5,
  '-1-10': 0.5,
  '-1-11': 0.5,
  '-10-1': 0.5,
  '-100': 0.5,
  '-101': 0.5,
  '-11-1': 0.5,
  '-110': 0.5,
  '-111': 0.5,
  '0-1-1': 0.5,
  '0-10': 0.5,
  '0-11': 0.5,
  '00-1': 29.19}}

In [60]:
path_dic

{'VABJIS_clean': {'-1-1-1': 5.93,
  '-1-10': 5.99,
  '-1-11': 5.91,
  '-10-1': 14.43,
  '-100': 10.74,
  '-101': 13.09,
  '-11-1': 4.91,
  '-110': 6.98,
  '-111': 6.97,
  '0-1-1': 13.09,
  '0-10': 10.74,
  '0-11': 14.43,
  '00-1': 29.19}}

In [72]:
path_dic

{'VABJIS_clean': {'-1-1-1': 10.59,
  '-1-10': 10.56,
  '-1-11': 10.56,
  '-10-1': 9.72,
  '-100': 13.01,
  '-101': 9.87,
  '-11-1': 11.49,
  '-110': 11.54,
  '-111': 11.58,
  '0-1-1': 9.89,
  '0-10': 13.04,
  '0-11': 9.73,
  '00-1': 29.19}}

In [52]:
larger_node = []
name_larger_node = []
for node in final.nodes:
    larger_node.append(final.nodes[node]['rad_max_sph'])
    name_larger_node.append(node)
node_largest = name_larger_node[np.argmax(larger_node)]
node_smallest = name_larger_node[np.argmin(larger_node)]
final.nodes[node_largest], final.nodes[node_smallest]
node_largest

20

In [10]:
5142 - key_plus*int(5142/key_plus)

176

In [21]:
a, b = dijkstra(176, 5142, {}, final)

In [24]:
b[5142]

KeyError: 5142

In [16]:
pos_order['000']

13

In [17]:
P[5142]

KeyError: 5142

In [19]:
D[5142]

inf

In [6]:
final2.nodes

NodeView((0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 211, 212, 214, 215, 216, 218, 220, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 235, 236, 2

In [95]:



# num_cores = multiprocessing.cpu_count()
# # mat_dic = {}

# path_dic = {}

# pool = multiprocessing.Pool(num_cores)


# # Step 2: `pool.apply` the `howmany_within_range()`
# volumen_res = pool.map(Diffusion2, os.listdir('./Nt2Files_MOF')[3:5])



# # Step 3: Don't forget to close
# pool.close()


def Diffusion2(file):

    
    

    nt2_name = file

    vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt = CellLocation(nt2_name)
    G, connection_cell, larger_node_dic = GraphCell(nt2_name, pos_order)

    final, key_plus = GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh)
    Y, Y1 = TransfMatrix(vector_values_latt)
    final2 = distance(final,3, Y, Y1)



    neigh_dic = {}
    for n, nbrs in final2.adj.items():
        neigh = []
        for nbr, eattr in nbrs.items():
            neigh.append(nbr)


        neigh_dic[str(n)] = neigh

    all_path = []
    total_path = []
    special_list = list(range(13))

    node_path = mat_pr_dic[nt2_name.split('.')[0]]

    for location in special_list:
        parents = {}

        start = node_path + key_plus*location

        target = key_plus*pos_order['000'] + node_path

        D, P = dijkstra(start, target, parents, final2)

        path = backpedal(start, target, P)

        distance_path = []
        for i in range(len(path)-1):

            distance_path.append(final2[path[i]][path[i+1]]['distance'])


        all_path.append(path)
        total_path.append(sum(distance_path))  


    total_vol = []
    for ind_path in all_path:
        volume = []
        for i in range(len(ind_path) - 1):
            volume.append(
                final2[ind_path[i]][ind_path[i+1]]['distance']*(final2[ind_path[i]][ind_path[i+1]]['rad_max_sph']**2)*pi)
        total_vol.append(sum(volume))

    path_dic_mat = {}

    for j in range(len(list(pos_order.keys())[0:13])):
        path_dic_mat[str(list(pos_order.keys())[j])] = round(total_vol[j]/total_path[j], 2)


    path_dic[nt2_name.split('.')[0]] = path_dic_mat
    print(path_dic)
    return path_dic

def CellLocation(nt2_name):
    
    from pymatgen.core.structure import Structure
    from math import pi, cos, sin
    
    six_letter_code = nt2_name.split('.')[0]
    
    path_nt2_file = './Nt2Files_MOF/' + nt2_name
    path_cif_file = './MOF/' + six_letter_code + '.cif'
    
    cif_structure = Structure.from_file(path_cif_file)
    
    latt = cif_structure.lattice
    
    # Angles of the cell's axis
    alpha = latt.alpha
    beta = latt.beta
    gamma = latt.gamma

    # Constant to change from degrees to radians
    cte = pi/180

    # Vectors proyections of the cell's vector over the ortogonal axis
    x_axis = latt.a
    y_axis = latt.b
    z_axis = latt.c

    # Cell's vectors
    a, b, c = round(x_axis*sin(beta*cte), 3) , 0 ,round(x_axis*cos(beta*cte), 3)
    d, e, g = round(y_axis*cos(gamma*cte), 3) ,round(y_axis*sin(gamma*cte), 3), 0
    l, m, n = 0, round(z_axis*cos(alpha*cte), 3), round(z_axis*sin(alpha*cte), 3)
    
    vector_values_latt = (a, b, c, d, e, g, l, m, n)

    # Constant that limits the maximum number of cells that could expand over every direction 
    # (axis and diagonal direction)
    r = 1

    cubic_comb = []
    
    # Each of the cell's vectors represent the length of the cell in that direction.
    # Independently of which length it is if you multiply the length of one of the vector by two it is clear
    # that the result vector will give the information about the location of the second cell in the direction
    # of the vector we multiply by two.

    # We generate the location of the cells for a cube of length equals to len(range(-r, r + 1))
    # r = 1, len(range(-r, r + 1)) = 27
    # The information of location of the cells are saved in real coordinates using the combination of the 
    # cell's vectors and in an orthogonal base transformation using as vectors the numbers (i,j,k) we use to multiply 
    # the cell's vectors
    for i in range(-r, r+1):
        for j in range(-r, r+1):
            for k in range(-r, r+1):
                cubic_comb.append((np.array([a,b,c])*i + np.array([d,e,g])*j + np.array([l,m,n])*k,
                                   np.array([i,j,k])))

    vector_pos = []
    vector_ort = []
    # Unpack values
    # We also choose as a center the vector 0 
    
    #     print('This is the Lattice vector: {}'.format(lattice_vector.abc))

    for vec, ort in cubic_comb:
        vector_pos.append(list(vec))
        vector_ort.append(list(ort))

    # Later in this Notebook we are going to create a graph for every vector we generated in the previous step.
    # These graphs will be composed by nodes whose names will be an ordered number.
    # The names will be assigned by the order the graphs were generated.
    # That means every graph will have as the name of their nodes a range of numbers whose length will be equal 
    # to the name of the last node we create in the first graph.
    # In order to keep track of the order we create we are going to establish that order by numbered the orthogonal
    # vectors

    pos_order = {}
    i = 0
    for vector in vector_ort:
        name = str(vector[0]) + str(vector[1]) + str(vector[2]) 
        pos_order[name] = i
        i += 1

    twentysix_faces_neigh = [np.array(vector_name) for vector_name in vector_ort]
    
    return vector_pos, vector_ort, pos_order, twentysix_faces_neigh, vector_values_latt


def GraphCell(nt2_name, pos_order):
    
    import networkx as nx
    
    # Initialize a networkx's graph. 
    # We take the information about the nodes and edges from the .nt2 file and passes it out to the graph.

    # Every node in the graph will have three attributes.
    # -Cartesian coordinate
    # -Pagerank coefficient
    # -Radius of the maximus sphere that fit in that node

    # Evey edge will have one attribute:
    # -Radius of the maximus sphere that can travel through the edge

    # We are not going to save the information about edges that connect point from outside of the cell
    
    path_nt2_file = './Nt2Files_MOF/' + nt2_name
    
    G = nx.Graph()
    
    cell_names = list(pos_order.keys())
    
    connection_cell = {cell_name:[] for cell_name in cell_names}
    
    larger_dist_atom = []
    
    pagerank_init_value = 100
    with open(path_nt2_file) as f:
        line = ' '

        while line:
            line = f.readline()

            if line == 'Vertex table:\n': 

                while line:

                    line = f.readline()

                    if line == '\n':
                        break
                    else:
                        line_list = line.split()
                        key = int(line_list[0])
                        coord_x = float(line_list[1])
                        coord_y = float(line_list[2])
                        coord_z = float(line_list[3])
                        min_dist_atom = float(line_list[4])

                        G.add_node(key, coord=np.array([coord_x, coord_y, coord_z]), rad_max_sph=min_dist_atom,
                                  pagerank=pagerank_init_value)
                        larger_dist_atom.append(min_dist_atom)

            if line == 'Edge table:\n':


                while line:

                    line = f.readline()
                    if line == '':
                        break
                    line_list = line.split()
                    
                    origin = int(line_list[0])
                    destination = int(line_list[2])
                    
                    larger_radius = float(line_list[3])
                    
                    x_sim = int(line_list[4])
                    y_sim = int(line_list[5])
                    z_sim = int(line_list[6])
                    
                    identifier = str(x_sim) + str(y_sim) + str(z_sim)
                    
                    if sum([abs(x_sim), abs(y_sim), abs(z_sim)]) == 0:
                        G.add_edge(origin, destination, rad_max_sph=larger_radius)
                        

                    else:
                        connection_cell[identifier].append((origin,destination,larger_radius))

    
    larger_node = max(larger_dist_atom)
    index_larger_node = np.argmax(larger_dist_atom)
    name_larger_node = list(G.nodes)[index_larger_node]
    
    return G, connection_cell, {'LargerNode':larger_node, 'NameLargerNode':name_larger_node} 


def GraphMultiCell(G, vector_pos, vector_ort, pos_order, connection_cell, twentysix_faces_neigh):
    
    import networkx as nx
    from copy import deepcopy
    from networkx.relabel import relabel_nodes
    # Create a dictionary which contains one graph for every cell we want to use to calculate the 
    # pagerank coefficients. 
    # These new graphs are a copy of the one that we initialized in previous step so, we 
    # change the names and the cartesian coordinates for every node

    graph_mod= {}

    # As we mentioned earlier we are goint to name the nodes wit a number.
    # The name of the last node of the first graph plus one (because we start in 0) will give as the range in every 
    # graph.

    key_plus = sorted(list(G.nodes), reverse=True)[0] + 1

    # The names used as keys in the dictionary of graph are the ortogonal base transformations we calculated before
    # They names are very useful to know the position of the graph with respect to the origin (0,0,0).

    for i in range(len(vector_pos)):
        name = str(vector_ort[i][0])+str(vector_ort[i][1])+str(vector_ort[i][2])
        graph_mod[name] = deepcopy(G)

        mapping_name = {}
        multiplier = pos_order[name]
        for node in graph_mod[name].nodes:
            graph_mod[name].nodes[node]['coord'] += vector_pos[i] #*box
            mapping_name[node] = node + key_plus*multiplier
        relabel_nodes(graph_mod[name], mapping=mapping_name, copy=False)

    # As you can see here it is really to see where the graph is if we now their name
    #     print('Keys of the Dictionary of Graph: {}'.format(graph_mod.keys()))


    # List with the names of the graph
    
    def symm(array):
    
        import numpy as np
        first = np.where(array == 2, -1, array)
        second = np.where(first == -2, 1, first)

        return second 

    # Now we add to the graph the edges that connect with nodes outside of the cell. We do this for every graph 
    # in the dictionary

    for j in range(len(vector_pos)):
        for neighbor in twentysix_faces_neigh:
            str_neighbor = str(neighbor[0]) + str(neighbor[1]) + str(neighbor[2]) 

            neig_loc = vector_ort[j] + neighbor
            str_neig_loc = str(neig_loc[0])+str(neig_loc[1])+str(neig_loc[2])

            if str_neig_loc in list(pos_order.keys()):
                name = str(vector_ort[j][0])+str(vector_ort[j][1])+str(vector_ort[j][2])
                multiplier_origin = pos_order[name]
                multiplier_destination = pos_order[str_neig_loc]

                for edge in connection_cell[str_neighbor]:
                    edge_1 = edge[0] + multiplier_origin*key_plus
                    edge_2 = edge[1] + multiplier_destination*key_plus
                    rad_max_sph = edge[2]
                    graph_mod[name].add_edge(edge_1, edge_2)
                    graph_mod[name].edges[edge_1, edge_2]['rad_max_sph'] = rad_max_sph

            else:
                symm_vec = symm(neig_loc)

                name = str(vector_ort[j][0])+str(vector_ort[j][1])+str(vector_ort[j][2])
                str_neig_loc_symm = str(symm_vec[0])+str(symm_vec[1])+str(symm_vec[2])

                multiplier_origin = pos_order[name]
                multiplier_destination = pos_order[str_neig_loc_symm]

                for edge in connection_cell[str_neighbor]:
                    edge_1 = edge[0] + multiplier_origin*key_plus
                    edge_2 = edge[1] + multiplier_destination*key_plus
                    rad_max_sph = edge[2]
                    graph_mod[name].add_edge(edge_1, edge_2)
                    graph_mod[name].edges[edge_1, edge_2]['rad_max_sph'] = rad_max_sph

    final = nx.Graph()

    for key in graph_mod:
        final.add_nodes_from(graph_mod[key])
        final.add_edges_from(graph_mod[key].edges)
        for node in graph_mod[key].nodes:
            for attr in graph_mod[key].nodes[node]:
                final.nodes[node][attr] = graph_mod[key].nodes[node][attr]

        for edge in graph_mod[key].edges:
            for attr in graph_mod[key].edges[edge[0], edge[1]]:
                final.edges[edge][attr] = graph_mod[key].edges[edge][attr]
                
    return final, key_plus

def TransfMatrix(vector_values_latt):
    import numpy as np
    a,b,c,d,e,g,l,m,n = vector_values_latt
    # Matrix to change the orthogonal base vector to the base vector of the unit cell

    A = np.array([[a,b,c],[d,e,g], [l,m,n]]).transpose()
    B1 = np.array([1, 0, 0])
    B2 = np.array([0, 1, 0])
    B3 = np.array([0, 0, 1])
    X = np.array([np.linalg.inv(A).dot(B1),np.linalg.inv(A).dot(B2),np.linalg.inv(A).dot(B3)])
    Y = X.transpose()

    # Matrix to change the base vector of the unit cell for the orthogonal base vector
    A1 = np.array([[1,0,0], [0,1,0], [0,0,1]]).transpose()
    B4 = np.array([a,b,c])#*3
    B5 = np.array([d,e,g])#*3
    B6 = np.array([l,m,n])#*3
    X1 = np.array([np.linalg.inv(A1).dot(B4),np.linalg.inv(A1).dot(B5),np.linalg.inv(A1).dot(B6)])
    Y1 = X1.transpose()

    return Y, Y1

def distance(final, multi_box, Y, Y1):
    # Function for the calculation of distances between axes
    import numpy as np
    dim = 3
    real_box = np.array([multi_box,multi_box,multi_box])
    
    for edge in final.edges:
        p1 = Y.dot(final.nodes[edge[0]]['coord'])
        p2 = Y.dot(final.nodes[edge[1]]['coord'])
    #     print(p1, p2)
        dist_3d = np.zeros((dim))
        for j in range(dim):
            dist = abs(p2[j]-p1[j])
            if dist > real_box[j] * 0.5:
                dist = abs(real_box[j] - dist)
            dist_3d[j] = dist
        
        final[edge[0]][edge[1]]['distance'] = np.sqrt(np.sum(np.power(Y1.dot(dist_3d), 2)))
             #- (final.nodes[edge[0]]['rad_max_sph'] + final.nodes[edge[1]]['rad_max_sph'])
    return final

# for edge in final.edges:
#     final[edge[0]][edge[1]]['distance'] = distance(final, edge[0], edge[1], 3, Y, Y1)



def dijkstra(start_vertex, target, parents, final, tol):
    from queue import PriorityQueue
    
    # Dictionary with keys = vertices and values = weights of the vertices (set as infinity at the start)
    D = {}
    visited = []
    node_list = list(final.nodes)

    D = {v:float('inf') for v in node_list}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        if current_vertex == target:
            print('Target vertex: ' + str(target) + ' reached')
            break
        else:

            visited.append(current_vertex)


            # Loop for every vertex of the net
            for neighbor in neigh_dic[str(current_vertex)]:
#                         if self.edges[current_vertex][neighbor] != -1:
               
                distance_edge = final[current_vertex][neighbor]['distance']

                if neighbor not in visited:
                    old_cost = D[neighbor]
                    new_cost = D[current_vertex] + distance_edge
                    if (new_cost < old_cost and (final.nodes[neighbor]['rad_max_sph']>tol and final[current_vertex][neighbor]['rad_max_sph']>tol)):
                        pq.put((new_cost, neighbor))
                        D[neighbor] = new_cost
                        parents[neighbor] = current_vertex

    return D, parents

def backpedal(start_vertex, target, searchResult):

    node = target

    backpath = [node]

    path = []

    while node != start_vertex:

        backpath.append(searchResult[node])

        node = searchResult[node]

    for i in range(len(backpath)):

        path.append(backpath[-i - 1])

    return path 



In [27]:
from queue import PriorityQueue
start_vertex = 176
target = 5142
parents = {}
# Dictionary with keys = vertices and values = weights of the vertices (set as infinity at the start)
D = {}
visited = []
node_list = list(final.nodes)

D = {v:float('inf') for v in node_list}
D[start_vertex] = 0

pq = PriorityQueue()
pq.put((0, start_vertex))

while not pq.empty():
    (dist, current_vertex) = pq.get()
    if current_vertex == target:
        print('Target vertex: ' + str(target) + ' reached')
        break
    else:
        print(current_vertex)
        visited.append(current_vertex)


        # Loop for every vertex of the net
        for neighbor in neigh_dic[str(current_vertex)]:
#                         if self.edges[current_vertex][neighbor] != -1:

            distance_edge = final[current_vertex][neighbor]['distance']

            if neighbor not in visited:
                old_cost = D[neighbor]
                new_cost = D[current_vertex] + distance_edge
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor))
                    D[neighbor] = new_cost
                    parents[neighbor] = current_vertex


176
948
186
174
942
1131
181
1133
182
151
190
1132
1127
138
139
908
7063
953
156
187
1116
7016
159
159
188
7061
136
185
185
173
125
124
1124
943
1123
7989
1122
1125
1129
7017
7011
7012
135
1130
947
141
1128
102
140
7994
163
980
1112
1134
1117
1110
7066
7050
1120
1109
3620
7013
137
4386
1101
7783
98
7990
914
1121
947
1108
7057
7993
130
1104
3624
157
1100
7015
1105
4380
197
1118
1111
128
3614
7052
978
93
7976
1107
3576
7057
1119
196
7047
909
96
7977
7062
7006
7064
7785
171
7072
3619
7072
1111
1119
1104
7073
4571
7982
1113
164
7785
7824
1056
1142
7033
197
909
196
1059
1145
7027
7049
191
1060
979
3589
3597
3566
907
7001
1057
912
7058
1103
7829
7040
1144
3612
4569
7818
7032
7790
1136
913
1051
3594
6996
4569
8007
1059
120
192
7035
1052
7000
906
4570
7782
1143
3562
7014
7992
4565
1021
7790
1024
8009
6992
3577
116
193
3563
3594
4391
1023
6978
7784
7039
911
1020
7900
7787
910
3540
8000
7786
4346
7819
1080
3628
7930
1020
7998
1140
8001
1141
194
3574
886
7762
7897
886
4350
7823
6974
1039
8008
357

10177
5901
10223
5703
2488
3301
9339
5531
6765
2433
2093
5926
2463
4755
2265
9141
9270
6847
10285
5926
5703
2463
9128
5690
8969
2394
3409
9332
2252
2456
2202
9078
10087
10104
10187
5640
2251
9127
10285
3409
5564
2205
9081
3311
5689
6847
2126
5643
5832
2206
9082
5894
5531
5644
2093
10127
9002
3255
2254
9130
3251
6693
6830
10268
5692
3392
10094
8969
10131
6831
10269
2197
9073
3393
10185
5635
2126
9002
2257
9133
3309
5564
5695
6637
10075
2198
9074
5867
3199
10082
3206
5636
9305
6644
9266
2429
2390
6739
2257
9133
6745
9298
6763
5695
6764
2422
5828
9365
2203
9079
9364
2489
6766
5641
2488
5860
2288
9164
6749
5726
2291
9167
9296
5729
5927
5562
2420
2420
9261
5926
2124
5858
2385
10077
3201
6639
6639
10077
9076
5638
3201
2200
9000
2249
9125
9264
9296
5823
5687
2388
2282
9158
5720
5826
2185
9061
6749
9077
5639
5623
2201
9071
5633
4775
6747
2195
1337
5563
9066
5628
6754
10192
2125
2159
9035
10080
3204
2190
3316
5597
6642
8213
10131
10081
3205
9001
6643
5728
2290
9063
5625
9075
5637
5850
10074
319

In [26]:
final[5142]

AtlasView({4768: {'rad_max_sph': 0.605995, 'distance': 0.4189999534779901}, 4951: {'rad_max_sph': 0.592359, 'distance': 0.8497313702011949}, 5140: {'rad_max_sph': 0.751209, 'distance': 0.7386831728758955}, 5152: {'rad_max_sph': 0.791739, 'distance': 0.7378504479357471}})

In [ ]:
final[edge[0]][edge[1]]['distance'] = np.sqrt(np.sum(np.power(Y1.dot(dist_3d), 2)))

In [32]:
(final.nodes[1]['rad_max_sph']**3)*(4/3)*pi

0.05225609141437991

In [40]:
((final[1][5]['distance'] - final.nodes[1]['rad_max_sph'] - final.nodes[5]['rad_max_sph'])*
 (final[1][5]['rad_max_sph']**2)*pi) + (final.nodes[1]['rad_max_sph']**3)*(4/3)*pi +(final.nodes[5]['rad_max_sph']**3)*(4/3)*pi

0.11261776320089582

In [41]:
((final[1][5]['distance'])*(final[1][5]['rad_max_sph']**2)*pi)

0.0659781074193123